# Exploration of Static Data


In [ ]:
# Imports
import sys

import pandas as pd
import seaborn as sns

sys.path.append("..")

In [ ]:
# Load mobility stops data
stops = pd.read_csv("../data/stops.csv", delimiter=";")
print(f"Loaded {len(stops)} rows and {len(stops.columns)} columns.")

stops.head(5)

From
[SBB](https://www.sbb.ch/en/tickets-offers/private-transport/car-parking/park-and-rail.html):
Park your car at one of around 600 P+Rail stations and pay on an hourly or daily
basis – it’s easiest directly in the P+Rail app. The quick and relaxed way to
travel to your destination by car and train.


In [53]:
# Extract only the P+R stops
is_pr = ~stops.parkrail_number.isna()
pr = stops[is_pr]

# Extract relevant columns
cols = {"Geopos": "geoloc", "Stop name": "station",
        "Station abbreviation": "station_abbr"}
pr = pr[list(cols.keys())].rename(columns=cols)

# Parse geoloc column to extract latitude and longitude


def parse_geoloc(x): return tuple(map(float, pd.Series(x.split(", "))))


geolocs = pr.geoloc.apply(parse_geoloc)
longitude, latitude = zip(*geolocs)
pr["longitude"] = longitude
pr["latitude"] = latitude

# Drop geoloc column
pr.drop(columns="geoloc", inplace=True)

# Drop duplicates (probably occur because some other mode of transport available at station)
pr.drop_duplicates(inplace=True)

# Display
display(pr)

,station,station_abbr,longitude,latitude
33,Muttenz,MU,47.533591,7.647894
34,Pratteln,PR,47.522669,7.690817
35,Frenkendorf-Füllinsdorf,FRE,47.501470,7.719111
36,Liestal,LST,47.484461,7.731367
37,Lausen,LSN,47.470345,7.759763
...,...,...,...,...
790,Hünenberg Zythus,HUZY,47.173608,8.450095
791,Root D4,LBD,47.104825,8.373760
792,Bern Wankdorf,BNWD,46.967826,7.465478
795,Zug Schutzengel,ZGSE,47.173725,8.507242


In [55]:
# Save processed data
pr.to_csv("../data/pr_stations.csv", index=False)